## THIAW MOUHAMADOU LAMINE BARA

# Sequence Labelling

In this session we will build an HMM model for PoS-tagging and then CRF and neural models for Named Entity Recognition.

## Building a simple Hidden Markov Model

In this first part of the lab we will build a very simple bigram HMM using probability estimates over the Brown corpus, which is Part-of-Speech tagged.

Recall from course 6: probability estimates (with MLE estimation) can be calculated by dividing the number of occurrences of a bigram by the number of occurrences of the first word.

First of all, we import the corpus where we will estimate the probabilities:


In [1]:
from nltk.corpus import brown

This corpus is in the form of sequences of sentences, where each sentence is made by a sequence of pairs (word, POS-tag), like this:

In [2]:
brown.tagged_sents()

[[('The', 'AT'), ('Fulton', 'NP-TL'), ('County', 'NN-TL'), ('Grand', 'JJ-TL'), ('Jury', 'NN-TL'), ('said', 'VBD'), ('Friday', 'NR'), ('an', 'AT'), ('investigation', 'NN'), ('of', 'IN'), ("Atlanta's", 'NP$'), ('recent', 'JJ'), ('primary', 'NN'), ('election', 'NN'), ('produced', 'VBD'), ('``', '``'), ('no', 'AT'), ('evidence', 'NN'), ("''", "''"), ('that', 'CS'), ('any', 'DTI'), ('irregularities', 'NNS'), ('took', 'VBD'), ('place', 'NN'), ('.', '.')], [('The', 'AT'), ('jury', 'NN'), ('further', 'RBR'), ('said', 'VBD'), ('in', 'IN'), ('term-end', 'NN'), ('presentments', 'NNS'), ('that', 'CS'), ('the', 'AT'), ('City', 'NN-TL'), ('Executive', 'JJ-TL'), ('Committee', 'NN-TL'), (',', ','), ('which', 'WDT'), ('had', 'HVD'), ('over-all', 'JJ'), ('charge', 'NN'), ('of', 'IN'), ('the', 'AT'), ('election', 'NN'), (',', ','), ('``', '``'), ('deserves', 'VBZ'), ('the', 'AT'), ('praise', 'NN'), ('and', 'CC'), ('thanks', 'NNS'), ('of', 'IN'), ('the', 'AT'), ('City', 'NN-TL'), ('of', 'IN-TL'), ('Atlant

We recall (from the course) that a Hidden Markov Model is composed by:

- A set of $N$ states $Q = \{q_1, q_2, \ldots, q_N\}$
- A transition probability matrix $A=a_{11}\ldots a_{ij} \ldots a_{NN}$, where each $a_{ij}$ represents the probability of transitioning from state $q_i$ to $q_j$; note that $\sum_{j=1}^N{a_{ij}} = 1 \forall i$
- A sequence of $T$ observations $O = o_1, o_2, \ldots o_T$, each one drawn from a vocabulary of size $V=v_1, v_2, \ldots, v_M$, of size $M$;
- A sequence of *observation likelihoods* $B=b_i(o_t)$, also called **emission probabilities**, each expressing the probability of an observation $o_t$ being generated from a state $q_i$;
- Finally, an initial probability distribution $\Pi = \pi_i, \pi_2, \ldots, \pi_N$ where $\pi_i$ indicates the probability that the Markov chain will start from state $q_i$. Some states $q_j$ may have $\pi_j = 0$, meaning that they cannot be initial states. Also, $\sum_{i=1}^N{\pi_i}=1$.

In our case, the set of states $Q$ is made by the vocabulary of labels (the POS-tags). The vocabulary $V$ corresponds to the word vocabulary (i.e. all the set of different words that appear in our corpus). The observations correspond to the sentences in the corpus.

We will now split our corpus in a training and test set:



In [3]:
corpus = brown.tagged_sents()

training = corpus[:-10]
testing = corpus[-10:]

**Exercise 1**: Extract $Q$ and $V$ from the Brown corpus and determine their respective size

In [4]:
#insert your solution here
pos_tags= [tag for word, tag in brown.tagged_words()]
Q=set(pos_tags)
vocabulary= [word for word, tag in brown.tagged_words()]
V=set(vocabulary)
print(f"The size of Q in the corpus is {len(Q)}")
print(f"The size of V in the corpus is {len(V)}")

The size of Q in the corpus is 472
The size of V in the corpus is 56057


**Exercise 2**: Create the matrices ($A$, $B$ and $\Pi$) by using the probabilities estimated on the training set; since we are considering bigrams, the probabilities of the transition matrix will be calculated as $\frac{count(t_{-1}, t)}{count(t_{-1})}$.

*Important*: you will need to add smoothing (for instance Lidstone with $k=0.1$) on $B$ otherwise the output will be $0$.

In [5]:
#insert your solution here

#Expected output:
#pi matrix such that pi[i] is the probability of starting in state q_i
#A matrix (Q x Q sized) such that A[i][j] is the probability of moving from state q_i to state q_j
#B matrix (Q x O sized) such that B[i][j] is the probability of state q_i to emit the word (observation) o_j
Q=list(Q)
V=list(V)
import numpy as np
n=len(Q)
m=len(V)
k=0.1

A=np.zeros((n,n))
B=0.1*np.ones((n,m))
pi=np.zeros(n)
index_Q=dict()
index_V=dict()
for i in range(n):
    index_Q[Q[i]]=i
for i in range(m):
    index_V[V[i]]=i
for sentence in training:
    for i,sent in enumerate(sentence):
        q,v=index_Q[sent[1]],index_V[sent[0]]
        pi[q]+=1
        B[q][v]+=1
        if i<len(sentence)-1:
            q2=Q.index(sentence[i+1][1])
            A[q][q2]+=1

pi=pi/np.sum(pi)
A=A/np.sum(A,axis=1)[:,None]
B=B/np.sum(B,axis=1)[:,None]            



In [6]:
print(A)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


We have now a model and we can estimate its performance over the test set.

To do this, we need the Viterbi algorithm for the decoding. To help you, an implementation of Viterbi is provided:
(note: to use this version you need to assign a numeric id to each word and tag, if you haven't already)

In [7]:
"""
params is a triple (pi, A, B) where
pi = initial probability distribution over states
A = transition probability matrix
B = emission probability matrix

observations is the sequence of observations (in our case, the observed words)

the function returns the optimal sequence of states and its score
"""
def viterbi(params, observations):
    pi, A, B = params
    M = len(observations)
    S = pi.shape[0]

    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf') #cases that have not been treated
    backpointers = np.zeros((M, S), 'int')

    # base case
    alpha[0, :] = pi * B[:,observations[0]]

    # recursive case
    for t in range(1, M):
        for s2 in range(S):
            for s1 in range(S):
                score = alpha[t-1, s1] * A[s1, s2] * B[s2, observations[t]]
                if score > alpha[t, s2]:
                    alpha[t, s2] = score
                    backpointers[t, s2] = s1
    # now follow backpointers to resolve the state sequence
    ss = []
    ss.append(np.argmax(alpha[M-1,:]))
    for i in range(M-1, 0, -1):
        ss.append(backpointers[i, ss[-1]])

    return list(reversed(ss)), np.max(alpha[M-1,:])

#Example:

#original sentence: you can't very well sidle up to people on the street and ask if they want to buy a hot Bodhisattva .
#sentence as sequence of word indexes:
#word_index=[42350, 44913, 3024, 50638, 15858, 16209, 36949, 31092, 28334, 45518, 22719, 26179, 32651, 52996, 25205, 16840, 36949, 1402, 46003, 10606, 19795, 3739]

#predicted, score = viterbi((pi, A, B), word_index)

#predicted will be a sequence of tag indexes:
#[12, 55, 86, 39, 29, 4, 70, 7, 14, 7, 0, 6, 21, 28, 12, 55, 28, 27, 28, 0, 9, 14, 15]

In [8]:
#Example of results calculation


testing_formated = []
for sentence in testing:
    sent = ""
    words_index = [] #vector of word indices to be passed to Viterbi
    true_label= [] #vector of the true labels from labeled corpus
    for word,tag in sentence:
        words_index.append(index_V[word]) #word_to_index is a dictionary mapping a word to its index
        true_label.append(tag)
        sent=sent+" "+word
    testing_formated.append((words_index,true_label,sent))


for word_index,labels,sentence in testing_formated:
    print("  The sentence is:",sentence)
    print("   ##TRUE##    ##PRED##")
    predicted, score = viterbi((pi, A, B), word_index) #call the viterbi decoder
    for i,true_label in enumerate(labels):
        predicted_label = Q[predicted[i]] #Q here is the vector of tags, so that at Q[i] we have the i_th tag in literal form
        print("      "+true_label+"         "+predicted_label)


  The sentence is:  you can't very well sidle up to people on the street and ask if they want to buy a hot Bodhisattva .
   ##TRUE##    ##PRED##
      PPSS         PPSS
      MD*         MD*
      QL         QL
      RB         RB
      VB         VBD
      IN         RP
      IN         IN
      NNS         NNS
      IN         IN
      AT         AT
      NN         NN
      CC         CC
      VB         VB
      CS         CS
      PPSS         PPSS
      VB         VB
      TO         TO
      VB         VB
      AT         AT
      JJ         JJ
      NP         .
      .         .
  The sentence is:  Additionally , since you're going to be hors de combat pretty soon with sprue , yaws , Delhi boil , the Granville wilt , liver fluke , bilharziasis , and a host of other complications of the hex you've aroused , you mustn't expect to be lionized socially .
   ##TRUE##    ##PRED##
      RB         RB
      ,         ,
      CS         CS
      PPSS+BER         PPSS+BER
      VBG     

**Exercise 3**: calculate Precision, Recall and F-measure for the bigram model

In [9]:
true_labels=[]
predicted_label = []
for word_index,labels,sentence in testing_formated:
    true_labels.append(labels)
    predicted, score = viterbi((pi, A, B), word_index)
    to_append=[]
    for j,lab in enumerate(labels):
        to_append.append(Q[predicted[j]])
    predicted_label.append(to_append)

from sklearn.metrics import classification_report
   

In [10]:
#concatenate the list of labels to pass to the classification report  
print(classification_report(np.concatenate(true_labels),np.concatenate(predicted_label),zero_division=0))

              precision    recall  f1-score   support

          ''       0.00      0.00      0.00         0
           ,       1.00      1.00      1.00        24
          --       1.00      1.00      1.00         2
           .       0.88      1.00      0.93         7
          AP       1.00      1.00      1.00         4
          AT       0.96      1.00      0.98        26
       AT-HL       0.00      0.00      0.00         1
          BE       1.00      1.00      1.00         2
        BEDZ       1.00      1.00      1.00         5
         BEZ       1.00      1.00      1.00         1
          CC       1.00      1.00      1.00         7
          CS       0.88      1.00      0.93         7
          DO       1.00      1.00      1.00         1
       FW-IN       0.00      0.00      0.00         1
       FW-NN       0.00      0.00      0.00         1
       FW-RB       0.00      0.00      0.00         1
          IN       0.95      0.95      0.95        19
       IN-HL       0.00    

**Exercise 4**: modify your HMM to use trigrams instead of bigrams, and re-evaluate the results

In [11]:
#Let me modify the hmm to use trigrams
#i will first recode the viterbi function to accept a trigram model

def concatenation_string(a, b):
    return a + ' ' + b
new_corpus=np.concatenate(corpus)
Q=list(set([concatenation_string(new_corpus[ct][1], new_corpus[ct+1][1]) for ct in range(len(new_corpus)-1)]))



n=len(Q)
m=len(V)
k=0.1

A=np.zeros((n,n))
B=0.1*np.ones((n,m))
pi=np.zeros(n)
index_Q=dict()
index_V=dict()
for i in range(n):
    index_Q[Q[i]]=i
for i in range(m):
    index_V[V[i]]=i
for sentence in training:
    for i, s in enumerate(sentence[1:],1):
        previous_st = sentence[i-1]
        q = index_Q[concatenation_string(previous_st[1], s[1])]
        v = index_V[s[0]]    
        pi[q] += 1
        B[q][v] += 1
        if i < len(sentence)-1:
            s_next = sentence[i+1]
            q_next = Q.index(concatenation_string(s[1], s_next[1]))
            A[q][q_next] += 1

pi=pi/np.sum(pi)
A=A/np.sum(A,axis=1)[:, np.newaxis]
B=B/np.sum(B,axis=1)[:, np.newaxis]            



p=2
def viterbi_trigram(params, observations):
    pi, A, B = params
    M = len(observations)
    S = pi.shape[0]

    alpha = np.zeros((M, S))
    alpha[:,:] = float('-inf') #cases that have not been treated
    backpointers = np.zeros((M, S), 'int')

    # base case
    alpha[0, :] = pi * B[:,observations[0]]
    p_likely_states = np.argsort(-alpha[0, :])[:p]


    # recursive case
    for t in range(1, M):
        for s2 in range(S):
            for s1 in p_likely_states:
                score = alpha[t-1, s1] * A[s1, s2] * B[s2, observations[t]]
                if score > alpha[t, s2]:
                    alpha[t, s2] = score
                    backpointers[t, s2] = s1
    # now follow backpointers to resolve the state sequence
    ss = []
    ss.append(np.argmax(alpha[M-1,:]))
    for i in range(M-1, 0, -1):
        ss.append(backpointers[i, ss[-1]])

    return list(reversed(ss)), np.max(alpha[M-1,:])

testing_formated = []
for sentence in testing:
    sent = ""
    words_index = [] #vector of word indices to be passed to Viterbi
    true_label= [] #vector of the true labels from labeled corpus
    for word,tag in sentence:
        words_index.append(index_V[word]) #word_to_index is a dictionary mapping a word to its index
        true_label.append(tag)
        sent=sent+" "+word
    testing_formated.append((words_index,true_label,sent))



for word_index,labels,sentence in testing_formated:
    print("  The sentence is:",sentence)
    print("   ##TRUE##    ##PRED##")
    predicted, score = viterbi_trigram((pi, A, B), word_index) 
    for i,true_label in enumerate(labels):
        predicted_label = Q[predicted[i]].split(" ")[1] 
        print("      "+true_label+"         "+predicted_label)


C:\Users\mouha\AppData\Local\Temp\ipykernel_21360\4177155655.py:37: RuntimeWarning: invalid value encountered in divide
  A=A/np.sum(A,axis=1)[:, np.newaxis]


  The sentence is:  you can't very well sidle up to people on the street and ask if they want to buy a hot Bodhisattva .
   ##TRUE##    ##PRED##
      PPSS         PPSS
      MD*         PPSS
      QL         PPSS
      RB         PPSS
      VB         PPSS
      IN         PPSS
      IN         PPSS
      NNS         PPSS
      IN         PPSS
      AT         PPSS
      NN         PPSS
      CC         PPSS
      VB         PPSS
      CS         PPSS
      PPSS         PPSS
      VB         PPSS
      TO         PPSS
      VB         PPSS
      AT         PPSS
      JJ         PPSS
      NP         PPSS
      .         AP
  The sentence is:  Additionally , since you're going to be hors de combat pretty soon with sprue , yaws , Delhi boil , the Granville wilt , liver fluke , bilharziasis , and a host of other complications of the hex you've aroused , you mustn't expect to be lionized socially .
   ##TRUE##    ##PRED##
      RB         AT
      ,         NN
      CS         NN
      PP

In [12]:
true_labels=[]
predicted_label = []
for word_index,labels,sentence in testing_formated:
    true_labels.append(labels)
    predicted, score = viterbi_trigram((pi, A, B), word_index)
    to_append=[]
    for j,lab in enumerate(labels):
        to_append.append(Q[predicted[j]].split(" ")[1])
    predicted_label.append(to_append)

from sklearn.metrics import classification_report
print(classification_report(np.concatenate(true_labels),np.concatenate(predicted_label),zero_division=0))
   

              precision    recall  f1-score   support

           ,       0.50      0.04      0.08        24
          --       0.00      0.00      0.00         2
           .       1.00      0.14      0.25         7
          AP       0.00      0.00      0.00         4
          AT       0.14      0.19      0.16        26
       AT-HL       0.00      0.00      0.00         1
          BE       0.00      0.00      0.00         2
        BEDZ       0.00      0.00      0.00         5
         BEZ       0.00      0.00      0.00         1
          CC       0.00      0.00      0.00         7
          CS       0.10      0.14      0.12         7
          DO       0.00      0.00      0.00         1
       FW-IN       0.00      0.00      0.00         1
       FW-NN       0.00      0.00      0.00         1
       FW-RB       0.00      0.00      0.00         1
          IN       0.17      0.21      0.19        19
       IN-HL       0.00      0.00      0.00         1
          JJ       0.00    

## Using NLTK's HMM implementation

We will compare now our model built from scratch to the implementation provided by NLTK:

In [28]:
import nltk
from nltk.tag import hmm

trainer = hmm.HiddenMarkovModelTrainer(states = Q, symbols = V)

model = trainer.train_supervised(training, estimator=lambda fd, bins: hmm.LidstoneProbDist(fd, 0.1, bins))
true_labels=[]
preditions=[]
for sent in testing:
    u_sent=[]
    tags=[]
    for word, tag in sent:
        u_sent.append(word)
        tags.append(tag)
    tagged=model.tag(u_sent)
    print(sent)
    print(tagged)
    true_labels.append(tags)
    prd=[t[1] for t in tagged]
    preditions.append(prd)



[('you', 'PPSS'), ("can't", 'MD*'), ('very', 'QL'), ('well', 'RB'), ('sidle', 'VB'), ('up', 'IN'), ('to', 'IN'), ('people', 'NNS'), ('on', 'IN'), ('the', 'AT'), ('street', 'NN'), ('and', 'CC'), ('ask', 'VB'), ('if', 'CS'), ('they', 'PPSS'), ('want', 'VB'), ('to', 'TO'), ('buy', 'VB'), ('a', 'AT'), ('hot', 'JJ'), ('Bodhisattva', 'NP'), ('.', '.')]
[('you', 'PPSS'), ("can't", 'MD*'), ('very', 'QL'), ('well', 'RB'), ('sidle', 'VBD'), ('up', 'RP'), ('to', 'IN'), ('people', 'NNS'), ('on', 'IN'), ('the', 'AT'), ('street', 'NN'), ('and', 'CC'), ('ask', 'VB'), ('if', 'CS'), ('they', 'PPSS'), ('want', 'VB'), ('to', 'TO'), ('buy', 'VB'), ('a', 'AT'), ('hot', 'JJ'), ('Bodhisattva', '.'), ('.', '.')]
[('Additionally', 'RB'), (',', ','), ('since', 'CS'), ("you're", 'PPSS+BER'), ('going', 'VBG'), ('to', 'TO'), ('be', 'BE'), ('hors', 'FW-RB'), ('de', 'FW-IN'), ('combat', 'FW-NN'), ('pretty', 'QL'), ('soon', 'RB'), ('with', 'IN'), ('sprue', 'NN'), (',', ','), ('yaws', 'NNS'), (',', ','), ('Delhi', 'NP

**Exercise 5**: Calculate precision, recall and F-measure and compare them to the results that you obtained with the two models (bigram and trigram) that you implemented before. Can you deduce whether the NLTK model is using bigrams or trigrams? (It is not stated in the manual)

In [29]:
print(classification_report(np.concatenate(true_labels), np.concatenate(preditions), zero_division=0))

              precision    recall  f1-score   support

          ''       0.00      0.00      0.00         0
           ,       1.00      1.00      1.00        24
          --       1.00      1.00      1.00         2
           .       0.88      1.00      0.93         7
          AP       1.00      1.00      1.00         4
          AT       0.96      1.00      0.98        26
       AT-HL       0.00      0.00      0.00         1
          BE       1.00      1.00      1.00         2
        BEDZ       1.00      1.00      1.00         5
         BEZ       1.00      1.00      1.00         1
          CC       1.00      1.00      1.00         7
          CS       1.00      1.00      1.00         7
          DO       1.00      1.00      1.00         1
       FW-IN       0.00      0.00      0.00         1
       FW-NN       0.00      0.00      0.00         1
       FW-RB       0.00      0.00      0.00         1
          IN       0.95      0.95      0.95        19
       IN-HL       0.00    

## Named Entity Recognition with Conditional Random Fields

For this exercise we will need to use the sklearn_crfsuite package. If it is not installed, it can be installed using pip with ```pip install sklearn-crfsuite```.

We will work on a Kaggle dataset named ```ner_dataset.csv``` (it should be in the same directory as the notebook).

Pandas can be used to read the content of the file:

In [30]:
import pandas as pd

data = pd.read_csv("ner_dataset.csv", encoding="latin1")
data = data.fillna(method="ffill") #repeat sentence number on each row

words = list(set(data["Word"].values)) #vocabulary V
n_words = len(words)

print(words[:10])
print(n_words)

C:\Users\mouha\AppData\Local\Temp\ipykernel_32012\3186707956.py:4: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data = data.fillna(method="ffill") #repeat sentence number on each row


['emerged', 'hardcourt', 'Rafah', ',', 'Athanase', 'Wasserhoevel', 'one-point-three', 'caps', 'Shon', 'Phyu']
35177


We provide you with some code that can read the sentences and produce the features in the format required by crf_suite. The ```SentenceGetter``` class transforms sentences into sequences of ```(word, POS, tag)``` triples

In [31]:
class SentenceGetter(object):

    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Word"].values.tolist(),
                                                           s["POS"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("Sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]

    def get_next(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
#load data
getter = SentenceGetter(data) #transform sentences into sequences of (Word, POS, Tag)
sentences = getter.sentences

C:\Users\mouha\AppData\Local\Temp\ipykernel_32012\985691969.py:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  self.grouped = self.data.groupby("Sentence #").apply(agg_func)


The next function allows us to define features that are used in the CRF. The features are stored in a dictionary.

In [32]:
def word2features(sent, i):
    """
    input:
       sent: sentence in the format of sequence of (Word, POS, Tag) triples
       i: position in the sentence
    output:
       features: a dictionary mapping the feature name into a value
    """
    word = sent[i][0]
    postag = sent[i][1]

    features = { #features related to the current position
        'bias': 1.0,
        'word.lower()': word.lower(),
        'postag': postag,
    }
    if i > 0: #features related to preceding word/tag
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True #feature for Beginning of Sentence

    if i < len(sent)-1: #features related to the following word/tag
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True #feature for end of sentence

    return features


def sent2features(sent):
    #transforms the sentence in a sequence of features
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    #transforms the sentence in a sequence of labels
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    #transforms the sentence in a sequence of tokens (removes POS tags and labels)
    return [token for token, postag, label in sent]


We can now build the features and label vectors, and create a CRF model:

In [33]:
!pip install sklearn_crfsuite


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

from sklearn_crfsuite import CRF
crf = CRF(algorithm='lbfgs',  max_iterations=100)


This will create a model with gradient descent algorithm ("lbfgs") and a limit of $100$ iterations.

Now we build the model and evaluate it on a 66/33 split between training and testing:

In [35]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
crf.fit(X_train, y_train)
pred=crf.predict(X_test)

n_pred = [item for sublist in pred for item in sublist]
n_test = [item for sublist in y_test for item in sublist]

report = classification_report(y_pred=n_pred, y_true=n_test)
print(report)


              precision    recall  f1-score   support

       B-art       0.40      0.01      0.03       137
       B-eve       0.69      0.31      0.42       111
       B-geo       0.83      0.91      0.87     12357
       B-gpe       0.98      0.82      0.89      5226
       B-nat       0.60      0.09      0.15        69
       B-org       0.78      0.67      0.72      6762
       B-per       0.83      0.79      0.81      5649
       B-tim       0.93      0.84      0.88      6650
       I-art       0.38      0.02      0.05       124
       I-eve       0.56      0.21      0.31        89
       I-geo       0.80      0.79      0.80      2433
       I-gpe       0.95      0.33      0.49        55
       I-nat       0.33      0.05      0.08        21
       I-org       0.76      0.78      0.77      5545
       I-per       0.85      0.87      0.86      5730
       I-tim       0.81      0.74      0.78      2110
           O       0.99      0.99      0.99    292571

    accuracy              

The report shows accuracy stats for all classes, but we are not interested in the **O** class. We can see that the scores for people names, place names and organizations are relatively low.

**Exercise 6**: Can you think of some new features for the CRF model to improve the results, especially on **B-org** ? Modify the *word2features* function to include the additional features and compare with the above results.

To improve results on B-org we can check if the word start with capital letter or is fully written with like in names  

In [36]:
def word2features(sent, i):
    """
    input:
       sent: sentence in the format of sequence of (Word, POS, Tag) triples
       i: position in the sentence
    output:
       features: a dictionary mapping the feature name into a value
    """
    word = sent[i][0]
    postag = sent[i][1]

    features = { #features related to the current position
        'bias': 1.0,
        'word.lower()': word.lower(),
        'postag': postag,
        'start_capital': word[0].isupper(),
        'all_caps': word.isupper(),
    }
    if i > 0: #features related to preceding word/tag
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:postag': postag1,
        })
    else:
        features['BOS'] = True #feature for Beginning of Sentence

    if i < len(sent)-1: #features related to the following word/tag
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:postag': postag1,
        })
    else:
        features['EOS'] = True #feature for end of sentence

    return features

def sent2features(sent):
    #transforms the sentence in a sequence of features
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    #transforms the sentence in a sequence of labels
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    #transforms the sentence in a sequence of tokens (removes POS tags and labels)
    return [token for token, postag, label in sent]


In [37]:
X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]

from sklearn_crfsuite import CRF
crf = CRF(algorithm='lbfgs',  max_iterations=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
crf.fit(X_train, y_train)
pred=crf.predict(X_test)

n_pred = [item for sublist in pred for item in sublist]
n_test = [item for sublist in y_test for item in sublist]

report = classification_report(y_pred=n_pred, y_true=n_test)
print(report)


c:\Users\mouha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\mouha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

       B-art       0.24      0.03      0.05       137
       B-eve       0.65      0.32      0.42       111
       B-geo       0.83      0.91      0.87     12357
       B-gpe       0.95      0.90      0.93      5226
       B-nat       0.67      0.03      0.06        69
       B-org       0.79      0.69      0.74      6762
       B-per       0.83      0.79      0.81      5649
       B-tim       0.93      0.84      0.88      6650
       I-art       0.31      0.04      0.07       124
       I-eve       0.50      0.21      0.30        89
       I-geo       0.81      0.78      0.79      2433
       I-gpe       0.96      0.45      0.62        55
       I-nat       0.00      0.00      0.00        21
       I-org       0.77      0.80      0.78      5545
       I-per       0.83      0.91      0.87      5730
       I-tim       0.81      0.74      0.77      2110
           O       0.99      0.99      0.99    292571

    accuracy              

c:\Users\mouha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## NER using a LSTM model

In this final section we will see an example of a neural network model written in PyTorch that uses a LSTM-based architecture for Named Entity Recognition.

First of all, we will prepare the data to have all information coded numerically (words and tags) and the sentences padded to a max length, in order to have all sentences of the same size.


In [38]:
#first of all we need to retrieve the number of different tags (a.k.a categories or classes of the words)
tags = list(set(data["Tag"].values))
n_tags=len(tags)

vocab = {w: i + 1 for i, w in enumerate(words)} #map words into a number
tag_map = {t: i for i, t in enumerate(tags)} #map tags into a number

from keras.preprocessing.sequence import pad_sequences
max_len=75

X = [[vocab[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=n_words) #pad with special token PAD, with ID=n_words
y = [[tag_map[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=-1) # -1 is associated to the PAD token


Now we will load the data and split them into training and test. We set batch size at 32.

In [39]:
import torch
from NERDataset import NERDataset
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

ner_train = NERDataset(X_train, y_train)
ner_test = NERDataset(X_test, y_test)

trainloader = torch.utils.data.DataLoader(ner_train, batch_size=32, shuffle=True)

The LSTM model is defined here. We have an embedding that maps each word in a vector (embedding) of size 100, which is learnt from the dataset. The embedded sentence is fed to a LSTM layer of size 50. The output is transferred to a fully connected layer with *n_tags* output, one for each of the possible labels. The loss is a cross-entropy loss over all tokens (excluding the "pad" tokens).

In [40]:
import torch.nn as nn
import torch.nn.functional as F

vocab_size=n_words+1
embedding_dim=100
lstm_hidden_dim=50

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        #maps each token to an embedding_dim vector
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        #the LSTM takens embedded sentence
        self.lstm = nn.LSTM(embedding_dim, lstm_hidden_dim, batch_first=True)

        #fc layer transforms the output to give the final output layer
        self.fc = nn.Linear(lstm_hidden_dim, n_tags)
    
    def forward(self, s):
        #apply the embedding layer that maps each token to its embedding
        s = self.embedding(s)   # dim: batch_size x batch_max_len x embedding_dim
        
        #run the LSTM along the sentences of length batch_max_len. We discard the cell state as output
        s, _ = self.lstm(s)     # dim: batch_size x batch_max_len x lstm_hidden_dim                
        
        #reshape the Variable so that each row contains one token
        s = s.reshape(-1, s.shape[2])  # dim: batch_size*batch_max_len x lstm_hidden_dim
        
        #apply the fully connected layer and obtain the output for each token
        s = self.fc(s)          # dim: batch_size*batch_max_len x num_tags
        
        return F.log_softmax(s, dim=1)   # dim: batch_size*batch_max_len x num_tags
    
def loss_fn(outputs, labels):
    #reshape labels to give a flat vector of length batch_size*seq_len
    labels = labels.view(-1)  
    
    #discard 'PAD' tokens
    mask = (labels >= 0).float()
    
    #the number of tokens is the sum of elements in mask
    num_tokens = int(torch.sum(mask))
    
    #pick the values corresponding to labels and multiply by mask
    outputs = outputs[range(outputs.shape[0]), labels]*mask

    #cross entropy loss for all non 'PAD' tokens
    return -torch.sum(outputs)/num_tokens
    


In the following block we carry out the training over 5 epochs

In [41]:
network = Net()
# Initialize optimizer
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)

num_epochs=5
#Run the training loop for defined number of epochs
for epoch in range(0, num_epochs):
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    i=0
    for inputs, targets in trainloader:
        #print(inputs, targets)
        
        optimizer.zero_grad() # Zero the gradients
        outputs = network(inputs) # Perform forward pass
        loss = loss_fn(outputs, targets) # Compute loss
        loss.backward() # Backprop
        optimizer.step() # Optimization
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0
        i+=1

Starting epoch 1
Loss after mini-batch   500: 1.678
Loss after mini-batch  1000: 0.844
Starting epoch 2
Loss after mini-batch   500: 0.750
Loss after mini-batch  1000: 0.687
Starting epoch 3
Loss after mini-batch   500: 0.624
Loss after mini-batch  1000: 0.580
Starting epoch 4
Loss after mini-batch   500: 0.535
Loss after mini-batch  1000: 0.497
Starting epoch 5
Loss after mini-batch   500: 0.460
Loss after mini-batch  1000: 0.431


**Exercise 7**: Evaluate the model over the test set (see the data preparation cells), compare the result with the previously seen CRF model

In [42]:
testloader = torch.utils.data.DataLoader(ner_test, batch_size=32, shuffle=True)
network.eval()
predicted = []
true_labels = []
for inputs, targets in testloader:
    outputs = network(inputs)
    predicted_tags = torch.argmax(outputs.view(-1, max_len, n_tags), 2)
    true_labels.append([tags[t] for t in targets.view(-1) if t != -1])
    predicted.append([tags[t] for t in predicted_tags.view(-1) if t != -1][:len(true_labels[-1])])

print(classification_report(np.concatenate(true_labels),np.concatenate(predicted),zero_division=0))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       137
       B-eve       0.00      0.00      0.00       111
       B-geo       0.09      0.03      0.04     12357
       B-gpe       0.08      0.01      0.02      5226
       B-nat       0.00      0.00      0.00        69
       B-org       0.16      0.00      0.00      6762
       B-per       0.08      0.01      0.02      5649
       B-tim       0.10      0.02      0.03      6650
       I-art       0.00      0.00      0.00       124
       I-eve       0.00      0.00      0.00        89
       I-geo       0.09      0.01      0.01      2433
       I-gpe       0.00      0.00      0.00        55
       I-nat       0.00      0.00      0.00        21
       I-org       0.13      0.01      0.01      5545
       I-per       0.08      0.02      0.03      5730
       I-tim       0.00      0.00      0.00      2110
           O       0.85      0.98      0.91    292571

    accuracy              

**Exercise 8**: Modify the model to use GloVe vectors as initial embeddings (hint: see the documentation of the <a href="https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html">Embedding</a> layer in PyTorch, in particular the *from_pretrained* method), and compare the results to the base one. 

In [43]:
from torchtext.vocab import GloVe
glove = GloVe(name="6B", dim = 100, max_vectors=vocab_size)


.vector_cache\glove.6B.zip: 862MB [10:57, 1.31MB/s]                               
100%|█████████▉| 35177/35178 [00:00<00:00, 41943.92it/s]


In [44]:

network=Net()
network.embedding = nn.Embedding.from_pretrained(glove.vectors)
optimizer = torch.optim.Adam(network.parameters(), lr=1e-4)


num_epochs=5
#Run the training loop for defined number of epochs
for epoch in range(0, num_epochs):
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    
    i=0
    for inputs, targets in trainloader:
        #print(inputs, targets)
        
        optimizer.zero_grad() # Zero the gradients
        outputs = network(inputs) # Perform forward pass
        loss = loss_fn(outputs, targets) # Compute loss
        loss.backward() # Backprop
        optimizer.step() # Optimization
        
        # Print statistics
        current_loss += loss.item()
        if i % 500 == 499:
            print('Loss after mini-batch %5d: %.3f' %
                  (i + 1, current_loss / 500))
            current_loss = 0.0
        i+=1

Starting epoch 1
Loss after mini-batch   500: 1.522
Loss after mini-batch  1000: 0.833
Starting epoch 2
Loss after mini-batch   500: 0.781
Loss after mini-batch  1000: 0.750
Starting epoch 3
Loss after mini-batch   500: 0.719
Loss after mini-batch  1000: 0.706
Starting epoch 4
Loss after mini-batch   500: 0.680
Loss after mini-batch  1000: 0.665
Starting epoch 5
Loss after mini-batch   500: 0.647
Loss after mini-batch  1000: 0.630


In [45]:
testloader = torch.utils.data.DataLoader(ner_test, batch_size=32, shuffle=True)
network.eval()
predicted = []
true_labels = []
for inputs, targets in testloader:
    outputs = network(inputs)
    predicted_tags = torch.argmax(outputs.view(-1, max_len, n_tags), 2)
    true_labels.append([tags[t] for t in targets.view(-1) if t != -1])
    predicted.append([tags[t] for t in predicted_tags.view(-1) if t != -1][:len(true_labels[-1])])

print(classification_report(np.concatenate(true_labels),np.concatenate(predicted),zero_division=0))

              precision    recall  f1-score   support

       B-art       0.00      0.00      0.00       137
       B-eve       0.00      0.00      0.00       111
       B-geo       0.14      0.00      0.01     12357
       B-gpe       0.00      0.00      0.00      5226
       B-nat       0.00      0.00      0.00        69
       B-org       0.00      0.00      0.00      6762
       B-per       0.12      0.00      0.01      5649
       B-tim       0.00      0.00      0.00      6650
       I-art       0.00      0.00      0.00       124
       I-eve       0.00      0.00      0.00        89
       I-geo       0.00      0.00      0.00      2433
       I-gpe       0.00      0.00      0.00        55
       I-nat       0.00      0.00      0.00        21
       I-org       0.00      0.00      0.00      5545
       I-per       0.00      0.00      0.00      5730
       I-tim       0.00      0.00      0.00      2110
           O       0.85      1.00      0.92    292571

    accuracy              

We have a little bit better accuracy